In [ ]:
import os
import numpy as np
import pandas as pd
import csv
import time
import shutil

# Gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import * 

from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/Code")
import cleaning_tweets as ct
os.chdir("/content/drive/MyDrive/Data")

Set text_id to the whole cleaned data set

In [ ]:
df_clean = pd.read_csv('cleaned_data.csv', header=None, dtype=str, encoding='utf-8-sig', engine='python')
df_clean.columns = ['date', 'id', 'text', 'hashtags', \
                                    'state', 'county and city',\
                                    'retweet_count', 'favorite_count']

df_id = pd.read_csv('textid.csv', header=None, dtype=str, encoding='utf-8-sig', engine='python')
df_id.columns = ["text", "text_id"]

df = pd.merge(df_clean, df_id, on='text', how="left")
df.dropna(subset=['text_id'], axis=0, how='any', inplace=True)
df.dropna(subset=['text'], axis=0, how='any', inplace=True)
df.drop_duplicates(['id'], keep='first', ignore_index=True, inplace=True)
df.reset_index(inplace=True, drop=True)

Assign sentiment labels and LDA results to cleaned data.

In [ ]:
df_st =  pd.read_csv('textid_st.csv', header = None, dtype = str, encoding = 'utf-8-sig', engine='python')
df_st.columns = ["text", "text_id", "st_labels", "st_scores"]
df_ap  = pd.read_csv('textid_ap.csv', header = None, dtype = str, encoding = 'utf-8-sig', engine='python')
df_ap.columns = ["text_id", "aspects"]

df_text = pd.merge(df_st, df_ap, on='text_id', how='left')
df_text.dropna(subset=['text_id'], axis=0, how='any', inplace=True)

tlst = list(df_text['text'].astype(str))
df_text.drop(labels=['text'], axis=1, inplace=True)

In [ ]:
new = pd.merge(df, df_text, on='text_id', how='left')
new.dropna(subset=['text_id'], axis=0, how='any', inplace=True)
new.drop_duplicates(['id'], keep='first', ignore_index=True, inplace=True)
new.reset_index(inplace=True, drop=True)
# new.to_csv('data_with_labels.csv', mode='a', header=None, index=False, encoding = 'utf-8-sig')

In [ ]:
df1 = new.copy()
df1.drop_duplicates(['text_id'], keep='first', ignore_index=True, inplace=True)
# remove tweets with empy value of "aspects"
df2 = df1.dropna(subset=['aspects'], axis=0, how='any').copy()
# df2.to_csv("data_with_labels_mini.csv", mode='a', index=False, header=None, encoding = 'utf-8-sig')
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4519697 entries, 0 to 4584928
Data columns (total 12 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   date             object
 1   id               object
 2   text             object
 3   hashtags         object
 4   state            object
 5   county and city  object
 6   retweet_count    object
 7   favorite_count   object
 8   text_id          object
 9   st_labels        object
 10  st_scores        object
 11  aspects          object
dtypes: object(12)
memory usage: 448.3+ MB


Extract aspects

In [ ]:
df2 = pd.read_csv('data_with_labels_mini.csv', header=None, dtype=str, encoding='utf-8-sig', engine='python')
df2.columns = ['date', 'id', 'text', 'hashtags', \
                            'state', 'county and city',\
                            'retweet_count', 'favorite_count',\
                            'text_id', 'st_labels', 'st_scores', 'aspects']

In [ ]:
# 4m data
topicidx_to_aspect, aspect_to_idx, idx_to_aspect = ct.getidx()

for idx, ap in idx_to_aspect.items():
    df2.loc[:, ap] = df2["aspects"].apply(lambda r: ct.dva(r, int(idx))[0])
    df2.loc[:, ap+'_score'] = df2["aspects"].apply(lambda r: ct.dva(r, int(idx))[1])

i = 0
for j in range(len(idx_to_aspect)):
    df_ap = df2.iloc[:, [0, 1, 2, 3, 4, 5, 8, 9, 10, 12+i, 13+i]].copy()
    ap_name = str(idx_to_aspect[str(j)])
    df_ap.dropna(subset=[ap_name], axis=0, how='any', inplace=True)
    df_ap.rename(columns={ap_name: 'aspects', ap_name+'_score': 'ap_scores'}, inplace=True)
    df_ap.loc[:, 'aspects_idx'] = [str(j)]*df_ap.shape[0]
    df_ap.reset_index(drop=True, inplace=True)

    df_ap.to_csv("ap_textid_mini_new.csv", header = None, mode='a', index=False, encoding = 'utf-8-sig')
    i += 2